In [1]:
from Selector import *
import sys
import time
from datetime import datetime
from view.HTML import show
import json
from indicators.MA import *
from indicators.TrendArea import *
from strategy.sequence.MAStrategy import *
from strategy.sequence.RStrategy import *
from simulation.Simulator import *
sys.path.append("../data")
import KDataDao as kds
from view.ViewDataFactory import *
from strategy.sequence.StrategyS1 import *


In [2]:
class KStrategy:
    """
    K线策略
    """
    def __init__(self):
        self.point = 0 
        self.m55 = MA(55)
        self.m10 = MA(10)
#         def rule(k):
#             return k.MOC()
        self.trend = TrendArea()
        self.condition = None
        self.segLow = 0
        self.buyDate = 0
        self.lmm = None
    def decide(self,list,trader,i):
        """
        策略
        :param list: 测试数据
        :param trader: 交易者
        :param i: 当前在list中的位置
        :return:
        """
        k = list[i]
        pre = list[i-1]
        self.trend.add(k)
        lm1 = self.trend.getMerge(-1)
        lm2 = self.trend.getMerge(-2)
        lm3 = self.trend.getMerge(-3)
        lm4 = self.trend.getMerge(-4)
        lm5 = self.trend.getMerge(-5)
        lm6 = self.trend.getMerge(-6)
        lm7 = self.trend.getMerge(-7)
        lm8 = self.trend.getMerge(-8)
        ls1 = self.trend.getSegment(-1)
        ls2 = self.trend.getSegment(-2)
        ls3 = self.trend.getSegment(-3)
        ls4 = self.trend.getSegment(-4)
        ls5 = self.trend.getSegment(-5)
        lb1 = self.trend.getBig(-1)
        lb2 = self.trend.getBig(-2)
        lb3 = self.trend.getBig(-3)
        lb4 = self.trend.getBig(-4)
        lb5 = self.trend.getBig(-5)
        la1 = self.trend.getMArea(-1)
        la2 = self.trend.getMArea(-2)
        la3 = self.trend.getMArea(-3)
        la4 = self.trend.getMArea(-4)
#         print "%s %s s:%s" % (datetime.fromtimestamp(lm1.ek.date/1000.0-24*60*60),lm1.day,ls1.day)
        #卖策略
        if trader.isHold(k.code) and  self.condition == 'a':
            if(la1.getSegSize() == 1 and k.close < la2.low):
                 #A1
                trader.sell(k.code,k.date,k.close,condition="a1")
                self.condition = None
                return
            return 
        if trader.isHold(k.code) :
            if k.close < self.point :
                trader.sell(k.code,k.date,k.close,condition="1")
                return
        if trader.isHold(k.code) and not ( lm1.low > lm2.low and (lm1.low-lm2.low)/lm2.low < 0.05 ): 
            if (
                list[i-2].ratio() < -0.04 and pre.isUp() and 
                pre.minOC() < list[i-2].close and 
                k.isDown() and 
                k.close < pre.minOC() 
               ) :
                #K
                trader.sell(k.code,k.date,k.close,condition="k1")
                return
            if (
                list[i-2].ratio() < -0.06 and pre.isUp() and 
                pre.maxOC() < list[i-2].MOC() and 
                k.isDown() and 
                k.open < pre.maxOC() 
               ) :
                #K
                trader.sell(k.code,k.date,k.close,condition="k2")
                return
            if (
                k.isDown() and pre.isUp() and
                k.close < pre.minOC() and
                k.close < list[i-2].minOC() and
                lm1.ratio < -0.03 and
                lm1.up < 2 and lm1.day > 4 and lm1.isDown() and
                (k.close > lm2.middle or k.close < lm2.klow)
               ) :
                #K
                trader.sell(k.code,k.date,k.close,condition="k3")
                return
            if (
                pre.ratio() < -0.075 and k.close < pre.MOC() 
               ) :
                #K
                trader.sell(k.code,k.date,k.close,condition="k4")
                return
            if(
                (lm2.ratio > 0.15 and k.close < pre.close and 
                 list[i-3].ratio() > 0.08 and 
                 k.isDown() and pre.isDown() and list[i-2].isDown())
            ):
                #K
                trader.sell(k.code,k.date,k.close,condition="k5")
                return
            if(
                k.ratio() < -0.07 and pre.ratio() > 0.05 and
                k.close < pre.open
            ):
                #K
                trader.sell(k.code,k.date,k.close,condition="k6")
                return
                
        if  trader.isHold(k.code) and self.condition == 5:   
            if k.ratio() < -0.03 or (k.ratio() < -0.02 and pre.ratio() < -0.02):
                #K
                self.condition = None
                trader.sell(k.code,k.date,k.close,condition="k20")
                return 
        if  trader.isHold(k.code) and self.condition != 0:
            if pre.ratio() < -0.08 and k.close < pre.MOC():
                trader.sell(k.code,k.date,k.close,condition="3")
                return
            if (
                k.isDown() and not pre.isUp() and 
                not list[i-2].isUp() and 
                (k.close - list[i-2].close)/list[i-2].close < -0.03
            ):
                trader.sell(k.code,k.date,k.close,condition="4")
                return
            if(
                (
                    lm1.high < lm2.high and
                    lm2.ratio < -0.20 and 
                    lm2.slope < -0.03 and
                    (k.ratio() < -0.02 or k.close < pre.close)  
                )
                or
                (
                    lm2.high < lm3.high and
                    lm3.ratio < -0.20 and 
                    lm3.slope < -0.02 and
                    lm2.ratio > 0.05 and
                    (k.isDown() or k.close < pre.close)
                )
                and self.buyDate < lm1.bk.date 

            ):
                #\
                # \|`
                trader.sell(k.code,k.date,k.close,condition="5")
                return
            
        if trader.isHold(k.code): 
            if(
                    self.buyDate < lm2.bk.date and
                    lm2.isUp() and 
                    k.close < lm2.middle and 
                    lm2.middle > lm3.high and
                    lm3.ratio < -0.07
                ):
                    #       /\
                    #     /   \
                    # /\/
                    self.condition = None
                    trader.sell(k.code,k.date,k.close,condition="8")
                    return
            if(
                    lm1.isDown() and  k.ratio() < -0.02 and
                    (lm2.ratio > 0.05 or lm2.day > 4) and
                    lm2.high < lm3.high and
                    lm2.low > lm5.high  


                ):
                    #     /\,~
                    # /\/   
                    self.condition = None
                    trader.sell(k.code,k.date,k.close,condition="9")
                    return
            if(
                    k.high < min(lm2.low,lm4.low,lm6.low) and
                    k.close > self.point
                ): 
                    self.condition = None
                    trader.sell(k.code,k.date,k.close,condition="10")
                    return
            if(
                    lb2.ratio > 0.2 and
                    ( (lb2.high > lb3.middle and lb2.high < lb3.high) or 
                      (lb2.high > lb5.middle and lb2.high < lb5.high)
                    ) and
                    lm2.high > lm3.middle and
                    lm3.high > lm2.middle and
                    k.MOC() < lm2.low and k.ratio() < -0.03
                ): 
                    #\
                    # \     /\/\
                    #  \  /     \
                    #   \/
                    #\
                    # \      /\/\
                    #  \    /    \
                    #   \/\/ 
                    self.condition = None
                    trader.sell(k.code,k.date,k.close,condition="11")
                    return
            if (
                (
                    (
                        (ls1.ratio > 0.08 and ls1.slope > 0.02) or 
                        (ls3.ratio >0.08 and ls3.slope > 0.02) 
                    ) and 
                    k.ratio() < 0.007 and pre.ratio() < 0.007 and k.close < pre.close
                )
                or
                (ls1.ratio > 0.15 and ls1.slope > 0.05 and k.ratio() < -0.01)
                or
                (ls2.day > 3 and ls2.slope > 0.04)
                or
                (lm1.ratio > 0.25 and pre.ratio() > 0.07 and k.ratio() < -0.02)
            ):
            # |`
            #|
                trader.sell(k.code,k.date,k.close,condition="12")
                return
#             if ls1.ratio < -0.04 and ls3.high > ls1.high and ls1.ratio < ls3.ratio :
#                 # \,
#                 #   \
#                 trader.sell(k.code,k.date,k.close,condition="13")
#                 return
#             if(lm1.ratio < -0.01 and k.close < lm2.high and 
#                abs(lm2.high - lm3.high)/lm3.high < 0.03 and lm3.ratio < -0.05):
#                 # /\/`
#                 trader.sell(k.code,k.date,k.close,condition="13b")
#                 return
                
        
        if trader.isHold(k.code) and self.condition == 0:
            if (
                (
                    lm2.khigh() < lm4.klow()  or
                    (k.high < lm4.klow() and 
                     k.high < lm2.klow() and 
                     lm2.khigh() < lm4.khigh() 
                     ) 
                ) and 
                lm4.bk.date > self.buyDate
            ):
                self.condition = None
                trader.sell(k.code,k.date,k.close,condition="14")
                return
            if(
                lb1 != None and
                ls2.ratio > 0.1 and 
                ls2.slope > 0.03 and 
                lb1.isUp() and lb2.isUp() and
                lb2 != lm4 and
                lb1.high > lb2.high 
            ): 
                self.condition = None
                trader.sell(k.code,k.date,k.close,condition="15")
                return
            if ( (k.high-self.point)/self.point < -0.01 ):
                self.condition = None
                trader.sell(k.code,k.date,k.close,condition="16")
                return
            if lm1.ratio > 0.4 :
                self.condition = None
        
        #买策略
        if lm1.slope > 0.06 and lm1.ratio > 0.4:
            self.lmm = lm1
        if self.lmm != None and k.close > self.lmm.middle:
            return;
        self.lmm = None
        if not trader.isHold(k.code):
            if(
                (
                    (
                        (lm2.ratio < -0.35 or lm2.day > 20) and
                        ls2.absR*2 < lm2.absR and
                        ls2.low < ls3.low and
                        (abs(ls3.absR-ls4.absR) < 0.02 or
                         abs(ls4.absR-ls5.absR) < 0.02 or
                         (min(ls3.low,ls5.low) > ls2.middle and  max(ls3.high,ls5.high) < lm2.middle ) or
                         ls3.day > 3 or ls3.ratio > 0.05) and
                        ls2.ratio < -0.05 and lm2.low < lm3.low
                    )
                    or
                    (
                        lm6.low > lm4.low and
                        lm2.ratio < -0.05 and
                        lm2.absR*1.5 < lm4.absR and
                        (lm2.low - lm4.low)/lm4.low < -0.04 
                    )
                    or
                    (
                        lm4.ratio < -0.2 and
                        lm2.ratio < -0.05 and
                        lm2.absR*1.5 < lm4.absR and
                        lm2.low < lm4.low
                    )
                    or(
                        lm4.day > 20 and
                        lm4.ratio < -0.1 and
                        lm2.ratio < -0.05 and
                        lm2.absR*1.5 < lm4.absR and
                        lm2.low < lm4.low
                    )
                )
                    and k.ratio() > 0.01 and lm1.day < 3
                    and (ls2.day > 3 or ls3.day > 3)

            ):
            # \
            #  \_
            #    \,
                self.point = min(lm1.low,k.minOC())
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='1a')
                return
            if(
                lb2.ratio < -0.3 and
                lm2.absR*2 < lb2.absR and
                (lm2.ratio < -0.1 or lm2.day > 5) and
                k.ratio() > 0.01 and lm1.day < 3

            ):
            # \
            #  \_
            #    \,
                self.point = min(lm1.low,k.low)
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='1b')
                return
            if(
                lm4.ratio < -0.1 and
                lm2.absR*1.2 < lm4.absR and
                lm4.day > lm2.day and
                lm2.low < lm4.low and
                k.ratio() > 0.01 and lm1.day < 4

            ):
            # \
            #  \_
            #    \,
                self.point = min(lm1.low,k.low)
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='1c')
                return
            if(
                lm6.ratio < -0.1 and
                lm2.absR < lm6.absR and
                (lm2.low - lm6.low)/lm6.low < -0.02 and
                (lm2.low - lm4.low)/lm4.low < -0.02 and
                lm2.ratio < -0.05 and
                lm6.middle > max(lm5.high,lm2.high) and
                lm1.day < 3
                and k.ratio() > 0.01
            ):
            #\
            # \
            #  \/\/\
            #       \
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='1d')
                return
            if(
                lm6.ratio < -0.1 and
                ls2.absR*2 < lm2.absR and
                lm2.middle < max(lm5.low,lm2.low) and
                lm6.middle > max(lm5.high,lm2.high) and
                lm1.day < 3
                and k.ratio() > 0.01
            ):
            # \
            #  \/\/\
            #       \
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='1f')
                return
            if(
                lm2.low < lm3.middle and
                lm3.high < lm4.middle and 
                ((lm4.absR > 0.1 and lm4.day > 5) or lm4.absR > 0.2) and
                (lm3.ratio > 0.1 or lm3.day > 7) and
                lm2.low > lm4.low and
                lm2.ratio < -0.04 and
                lm1.day < 3 and
                k.ratio() > 0.01 and
                lm2.absR*1.3 < lm4.absR and
                (ls2.slope > -0.04 or k.ratio() > 0.04)
            ):
            #\
            # \/\,
                self.point = min(lm1.klow,k.low)
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='2a')
                return
            if(
                lb3.isUp() and
                lb4.ratio < -0.4 and
                lb2.low < lb3.middle and
                k.low < lb3.middle and 
                (lb2.day > 3 or lb3.day > 3) and
                lb2.slope > -0.02 and
                ls2.slope > -0.02 and
                (lb2 > -0.15 or k.ratio() > 0.02) and
                lb1.day < 3 and
                k.isUp()
            ):
            #\
            # \/\,
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='2b')
                return
            if(
                lm2.low > lm4.low and
                lm4.absR*2 < lm6.absR and
                lm6.low > lm5.middle and
                lm4.low < lm6.low and
                lm1.day < 3 
#                 and
#                 (k.ratio() > 0.02 or (k.close - pre.close)/pre.close > 0.02)
            ):
            #\
            # \
            #  \/\ 
            #     \/\,
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='2c')
                return
            if(
                lm2.low < lm4.high and
                lm2.low > lm4.low and
                ( (lm4.low < lm6.high and lm4.low > lm6.low) or
                  (lm6.low < lm4.high and lm6.low > lm4.low and lm6.day > 3) ) and
                lm6.low > lm7.middle and
                lm1.day < 4
            ):
            #
            # 
            #  /\/\/\,
            #/   
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='2d')
                return
            if(
                lm2.slope > -0.015 and
                lm2.day > 5 and lm3.day > 5 and 
                lm3.mergeNum < 5 and 
                lm2.middle > max(lm4.high,lm6.high) and
                min(lm4.high,lm6.high) > max(lm4.low,lm6.low) and
                lm2.low > min(lm4.high,lm6.high) and
                lm7.low < min(lm4.low,lm6.low) and
                lm1.day < 3
            ):
            #        /\,
            #  /\/\/
            #/
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                trader.buy(k.code,k.date,k.close,condition='3a')
                return
            if(
                (lm2.ratio < -0.1 or (lm2.day > 20 and lm2.isDown()) ) and
                ls2.absR*2 < lm2.absR and
                ls2.day > 3 and
                lm2.high > max(lm4.high,lm6.high) and
                lm2.low > min(lm4.high,lm6.high) and
                lm1.day < 3 and 
                k.ratio() > 0.01
            ):
            #        /\,
            #  /\/\/    `
            #/
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                self.condition = 0
                trader.buy(k.code,k.date,k.close,condition='3b')
                return
          
        if k.isUp() and not trader.isHold(k.code):
            if(
                abs((lm2.low - lm6.low)/lm6.low) < 0.04 and
                lm3.low > lm6.low and
                lm2.low > lm6.low and
                max(lm2.absR,lm5.absR)*2 < lm6.absR and
                lm1.day < 3 and max(lm2.low,lm6.low) < lm7.middle and
                lm2.isDown()
            ):
                #\
                # \   /\/\
                #  \/     \,
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                trader.buy(k.code,k.date,k.close,condition='4')
                return
            if(
                lm6.ratio < -0.04 and 
                abs( max(lm5.high,lm2.high) - lm6.middle )/lm2.middle < 0.05 and
                lm2.low > lm6.low and lm3.low > lm6.low and
                lm2.ratio < -0.1 and lm2.slope < -0.03 and
                lm2.isDown()
            ):
                #\
                # \   /\/\
                #  \/     \,
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                trader.buy(k.code,k.date,k.close,condition='4')
                return
            if(
                lm2.isDown() and
                lm3.ratio > 0.06 and lm3.day > 3 and
                lm2.low > lm4.high and
                (lm2.low - lm4.high)/lm4.high < 0.05 and
                k.ratio() > 0.02 and
                lm1.day < 4
            ):
                #
                #     /\,  
                # /\/   
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
#                 self.condition = 5
                trader.buy(k.code,k.date,k.close,condition='5')
                return
            if(
                (
                    (
                        lm1.ratio < -0.25 and
                        (
                            (lm1.slope < -0.04 and lm1.mergeNum > 1) or 
                            (lm1.ratio < -0.4 and ls1.slope < -0.04) 
                        )
                        and k.ratio() > 0.02
                    ) 
                    or
                    (
                        lm2.ratio < -0.25 and
                        lm1.up < 3 and
                        (
                            (lm2.slope < -0.04 and lm2.mergeNum > 1 and k.ratio() > 0.02) or 
                            (lm2.ratio < -0.4 and lm2.slope < -0.02 and not k.isDown()) 
                        )
                        and lm1.ratio < 0.1
                    )
                ) 
                
                
            ):
                #\
                # \
                #  \,
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                trader.buy(k.code,k.date,k.close,condition='6')
                return
            if(
                k.low > pre.high and
                lm2.day > 3 and
                pre.close < lm2.high and
                k.close > lm2.high and
                (lm2.low > lm3.low or lm2.day < lm3.day)
            ):
                #
                #      ,
                # \/\/   
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                trader.buy(k.code,k.date,k.close,condition='7')
                return
            if(
                lm1.isUp() and
                k.close > lm2.high and
                pre.close < lm2.high and
                lm2.low > lm3.low and
                (lm2.low - lm3.low)/lm3.low < 0.03 and
                abs(lm4.low - lm5.low)/lm5.low < 0.03 
                
            ):
                #
                #        ,
                # /\/\/\/   
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
#                 self.condition = 5
                trader.buy(k.code,k.date,k.close,condition='8')
                return
        #区域分析
        if not trader.isHold(k.code):
            if (la1.getSegSize() == 1 and la2.low < la3.low and la2.ratio < la3.ratio and 
                la2.day*2 < la3.day and la2.getSegSize() > 4):
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                self.condition = 'a'
                trader.buy(k.code,k.date,k.close,condition='a1')
                return
            if (la1.getSegSize() == 1 and la2.low < la3.low and 
                    ( la2.ratio < la3.ratio or la2.day *2 < la3.day) and
                    k.close > la2.high and k.close > la3.high  
                ):
                self.point = min(lm1.klow(),k.low)
                self.buyDate = k.date
                self.condition = 'a'
                trader.buy(k.code,k.date,k.close,condition='a2')
                return
            
            

In [3]:
def firgueF(code):
    """显示股票"""
    clear_output()
    #获取数据
#     code = '002390'
#     code = '300051'
#     code = '002653'
    code = '002353'
    begin = "2013-06-05"
    end = "2016-11-07"
#     begin = "2016-03-27"
#     end = '2014-10-08'
    listK = kd.get_k_data(code,begin ,end)
    data,volume = ViewDataFactory.getCandlestick(listK)
    madic = ViewDataFactory.getMA(listK,(20,60))
    
    sk = KStrategy()
    sim = Simulator(sk)
    sim.runSingle(code,begin=begin,end=end)
    flags = ViewDataFactory.getTradeFlag(sim.trader.record,'#2AEBAF')
    earningsLine = sim.trader.earningsLine
    trendLine,trendMergeLine,trendBig = ViewDataFactory.trend2Line(sk.trend)
    areaRange,mareaRange = ViewDataFactory.trend2AreaRang(sk.trend,listK)
#     print areaRange
    
    mas = MAStrategy(60)
    sim = Simulator(mas)
    sim.runSingle(code,begin=begin,end=end)
    maFlags = ViewDataFactory.getTradeFlag(sim.trader.record,'#2AEBAF')
    maEarningsLine = sim.trader.earningsLine
    
    sim = Simulator(RStrategy())
    sim.runSingle(code,begin=begin,end=end)
    rLine = sim.trader.earningsLine
    #获取开始显示的时间
    mDate = 0
    if len(data)>0:
        if len(listK)/2>abs(-250):
            mDate = data[-250][0]
        else:
            mDate = data[len(listK)/2][0]
            
    #显示
    params = {}
    params["code"] = code
    params["mDate"] = mDate
    params["data"] = json.dumps(data)
    params["volume"] = json.dumps(volume)
    params["earningsLine"] = json.dumps(earningsLine)
    params["maEarningsLine"] = json.dumps(maEarningsLine)
    params["rLine"] = json.dumps(rLine)
    params["maFlags"] = json.dumps(maFlags)
    params["flags"] = json.dumps(flags)
    params["line20"] = json.dumps(madic[20].line)
    params["line60"] = json.dumps(madic[60].line)
    params["trendLine"] = json.dumps(trendLine)
    params["trendMergeLine"] = json.dumps(trendMergeLine)
    params["trendBig"] = json.dumps(trendBig)
    params["areaRange"] = json.dumps(mareaRange)
    
    show("StrategyArea.html",params)


In [4]:
class KB(Selector):
    """红绿红"""
    
    def analyze(self,list):
        """分析股票"""
        if True:
            return True
#         if (
#                         abs((list[-1].maxOC() - list[-3].maxOC()) / list[-3].maxOC()) < 0.03
#                 and list[-1].ratio() > 0.01
#             and list[-2].ratio() < -0.01
#         and list[-3].ratio() > 0.01):
#             return True
        mx,mn = HLRatio(len(list)-1,250,list)
        if mx > 1:
            return True
        return False
    
    def firgue(self,code):
        """显示股票"""
        firgueF(code)

In [5]:
kb = KB(False)
